In [61]:
import pandas as pd
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn as nn
from torchvision import models
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.optim as optim


In [97]:
full_label_df = pd.read_csv('selected_labels.csv') 
image_names = full_label_df['Image Number'].values # if you need it
# Now select only the label columns
label_df = full_label_df[['Start/Restar the game', 'The joy of the players', 'yellow card',
                          'Red Card', 'Stadium View', 'Penalty Image',
                          'Free Kick', 'Generic Moment']]

In [98]:
# Splits (based on the index of the full dataframe)
train_idx, test_idx = train_test_split(full_label_df.index, test_size=0.2, random_state=42)
train_idx, val_idx = train_test_split(train_idx, test_size=0.1, random_state=42)

train_df = label_df.loc[train_idx]
val_df = label_df.loc[val_idx]
test_df = label_df.loc[test_idx]

# Corresponding image names (important!)
train_img_names = full_label_df.loc[train_idx, 'Image Number'].values
val_img_names = full_label_df.loc[val_idx, 'Image Number'].values
test_img_names = full_label_df.loc[test_idx, 'Image Number'].values

In [100]:
# 3. Dataset and DataLoader
class FootballDataset(Dataset):
    def __init__(self, image_dir, label_df, image_names, transform=None):
        self.image_dir = image_dir
        self.label_df = label_df
        self.transform = transform
        self.image_names = image_names
        self.labels = label_df.values.astype('float32')

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = str(self.image_names[idx]) + '.jpg'
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(self.labels[idx])
        return image, label

In [101]:
# 5. Transforms and Loaders
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet means
                         [0.229, 0.224, 0.225])  # ImageNet stds
])

train_ds = FootballDataset('data/selected_images', train_df, train_img_names, transform)
val_ds = FootballDataset('data/selected_images', val_df, val_img_names, transform)
test_ds = FootballDataset('data/selected_images', test_df, test_img_names, transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)
test_loader = DataLoader(test_ds, batch_size=32)

In [102]:
# 6. Modify VGG13 for Multi-Label Classification
vgg13 = models.vgg13(pretrained=True)

# Freeze base layers (optional)
for param in vgg13.features.parameters():
    param.requires_grad = False

# Modify classifier to output 8 labels
vgg13.classifier[6] = nn.Sequential(
    nn.Linear(4096, 8),
    nn.Sigmoid()  # Sigmoid for multi-label classification
)

/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG13_Weights.IMAGENET1K_V1`. You can also use `weights=VGG13_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [103]:
# 7. Training Loop

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg13.to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(vgg13.parameters(), lr=1e-4)

def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [104]:
# 8. Evaluation

def evaluate(model, loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    return total_loss / len(loader)

In [105]:
for epoch in range(10):
    train_loss = train(vgg13, train_loader, optimizer, criterion)
    val_loss = evaluate(vgg13, val_loader)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")


Epoch 1, Train Loss: 0.1707, Val Loss: 0.1084
Epoch 2, Train Loss: 0.0695, Val Loss: 0.0883


In [ ]:
# 10. Inference on Test Set

def predict(model, loader, threshold=0.5):
    model.eval()
    all_preds = []
    with torch.no_grad():
        for images, _ in loader:
            images = images.to(device)
            outputs = model(images)
            preds = (outputs > threshold).cpu()
            all_preds.append(preds)
    return torch.cat(all_preds, dim=0)
